In [9]:
import pandas as pd
import os
import warnings
import joblib

os.chdir('/home/LULAB/wboohar/CODEX/data_processing/code')
from quantcell import quantcell_project
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier

Same parameters as in initialization.ipynb
 - annotations_path: a .json file corresponding to the marker combos, see GitHub or included sample_data for expected format
 - base_dir: folder where output folder will be created
 - project_name: name of output folder

In [ ]:
annotations_path = '/store/Projects/wboohar/PhenoCycler/annotation_strategies/marker_combos_062525_updated_verified.json'   
base_dir = '/store/Projects/wboohar/PhenoCycler' 
project_name = 'QuantCellPaperO3'
project_path = f'{base_dir}/{project_name}'

First we will initialize a QuantCell project object with the given parameters. It will load in expression data created by the initialization.ipynb notebook.

In [12]:
project = quantcell_project()
project.initialize(base_path=base_dir, project_name=project_name)

Initializing project QuantCellPaperO3 at /store/Projects/wboohar/PhenoCycler


Next, we preprocess the data for machine learning. See the paper for exact details.

In [13]:
project.process_data()

In [ ]:
if not os.path.exists(f'{project_path}/model_selection/RandomForestClassifier_best_params_macro.joblib'):
    # Handle the case where hyperparameter tuning is not done and use the base classifier
    best_params = {}
else:
    best_params = joblib.load(f'{project_path}/model_selection/RandomForestClassifier_best_params_macro.joblib')

classifier = RandomForestClassifier(
    **best_params,
    n_jobs=36,
    random_state=project.random_seed
)

project.set_clf(classifier)

In [16]:
project.fit()

Choose a false discovery rate (FDR) cutoff <= 1. 1 is equivalent to not thresholding at all and just labeling every cell with the highest probability cell type.

In [17]:
project.relabel_unannotated(FDR_cutoff=0.05)

Save the annotated cells to a .csv file in the output folder

In [22]:
project.overwrite_codex()
project.save_codex()